In [ ]:
import yaml
import os
import json
import gzip
import zlib
import datetime
from dateutil import parser
import multiprocessing
from subprocess import run, PIPE
import os
from pymongo import MongoClient
import pickle
import numpy as np
import scipy.sparse as sparse
client = MongoClient()
db = client.libs

In [ ]:
os.listdir('./libs')

In [ ]:
f= open('./libs/dependencies-1.0.0-2017-06-15.csv')
reader = csv.reader(f, delimiter=',')    

In [ ]:
next(reader)

In [ ]:
versions_heads

In [ ]:
for file in os.listdir('./libs'):
    if '.csv' in file:
        with open('./libs/{0}'.format(file)) as g:
            reader = csv.reader(g, delimiter=',')    
            heads = next(reader)
            example = next(reader)
            print('{0}: {1} \n {2} \n\n'.format(file, heads, example))

In this release you will find data about software distributed and/or crafted publicly on the Internet. You will find information about its development, its distribution and its relationship with other software included as a dependency. You will not find any information about the individuals who create and maintain these projects.

Further information and documentation on this data set can be found at https://libraries.io/data

For enquiries please contact data@libraries.io

This dataset contains seven files:

## Projects
A project is a piece of software available on any one of the 33 package managers supported by Libraries.io.

## Versions
A Libraries.io version is an immutable published version of a Project from a package manager. Not all package managers have a concept of publishing versions, often relying directly on tags/branches from a revision control tool.

## Tags
A tag is equivalent to a tag in a revision control system. Tags are sometimes used instead of Versions where a package manager does not use the concept of versions. Tags are often semantic version numbers.

## Dependencies
Dependencies describe the relationship between a project and the software it builds upon. Dependencies belong to Version. Each Version can have different sets of dependencies. Dependencies point at a specific Version or range of versions of other projects.

## Repositories
A Libraries.io repository represents a publicly accessible source code repository from either github.com, gitlab.com or bitbucket.org. Repositories are distinct from Projects, they are not distributed via a package manager and typically an application for end users rather than component to build upon.

## Repository dependencies
A repository dependency is a dependency upon a Version from a package manager has been specified in a manifest file, either as a manually added dependency committed by a user or listed as a generated dependency listed in a lockfile that has been automatically generated by a package manager and committed.

## Projects with related Repository fields
This is an alternative projects export that denormalizes a projects related source code repository inline to reduce the need to join between two data sets.

# License
This dataset is released under the Creative Commons Attribution-ShareAlike 4.0 International License.

This license provides the user with the freedom to use, adapt and redistribute this data. In return the user must publish any derivative work under a similarly open license, attributing Libraries.io as a data source. The full text of the license is included in the data.

# Access, Attribution and Citation
The dataset is available to download from Zenodo at https://zenodo.org/record/808273.

Please attribute Libraries.io as a data source by including the words ‘Includes data from Libraries.io’ and reference the Digital Object identifier: 10.5281/Zenodo.808273.

In [ ]:
with open('./libs/{0}'.format('dependencies-1.0.0-2017-06-15.csv')) as g:
        reader = csv.reader(g, delimiter=',')    
        headers = next(reader)
        print(headers)
        for entry in reader:
            e_dict = dict()
            for i, field in enumerate(entry):
                e_dict[header[i]] = field
            print(e_dict)
            break
#             db.dependencies.insert_one(e_dict)


In [ ]:
with open('./libs/{0}'.format('projects-1.0.0-2017-06-15.csv')) as g:
        reader = csv.reader(g, delimiter=',')    
        i=0
        for a in reader:
            print(a)
            break
            i+=1
        print(i)

### DataSet size:

projects: 2215454
dependencies: 51239987

In [ ]:
with open('./libs/{0}'.format('dependencies-1.0.0-2017-06-15.csv')) as g:
        reader = csv.reader(g, delimiter=',')    
        headers = next(reader)
#         db.dependencies.drop()
        thousand = 0
        t_list = list()
        for entry in reader:
            e_dict = dict()
            for i, field in enumerate(entry):
                e_dict[headers[i]] = field
            thousand += 1
            t_list.append(e_dict)
            if thousand % 1000 == 0:
                thousand=0
                
                db.dependencies.insert_many(t_list)
                t_list = list()
        
        db.dependencies.insert_many(t_list)



In [ ]:
with open('./libs/{0}'.format('projects-1.0.0-2017-06-15.csv')) as g:
        reader = csv.reader(g, delimiter=',')    
        headers = next(reader)
        db.projects.drop()
        thousand = 0
        t_list = list()
        for entry in reader:
            e_dict = dict()
            for i, field in enumerate(entry):
                e_dict[headers[i]] = field
            thousand += 1
            t_list.append(e_dict)
            if thousand % 1000 == 0:
                thousand=0
                
                db.projects.insert_many(t_list)
                t_list = list()
        
        db.projects.insert_many(t_list)
print(db.projects.count())

### things to do next

1. decide how to split the data to users and items
2. do filtering of dependencies
3. implement the simple MF
4. implement the recall or AUC

In [ ]:
db.dependencies.count()

In [ ]:
def data_iter(cursor, batch_size=10000):
    mylist = list()
    it = 0
    for item in cursor:
        it +=1
        mylist.append(item)
        if it%batch_size==0:
            yield mylist
            it=0
            mylist= list()



In [ ]:
deps_proj = dict()
proj_num = dict()

In [ ]:
cursor = db.dependencies.find()
batch_size=10000
cursor.batch_size(batch_size=batch_size)
my_iter = data_iter(cursor, batch_size)
for entry in cursor:
    deps_proj[entry['Project ID']] = deps_proj.get(entry['Project ID'],0) + 1
    dep_proj_name = entry['Dependency Project ID'] if entry.get('Dependency Project ID',False) else \
        (entry['Dependency Name']  + '_' +  entry['Dependency Kind']  + '_' +  entry['Dependency Platform']  \
        + '_' + entry['Dependency Project ID']  + '_' +  entry['Dependency Requirements'])
    proj_num[dep_proj_name] = proj_num.get(dep_proj_name, 0) + 1

### Dimensions

projects: 775580

Referenced projects: 420872

Total interactions: 51239986

In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
np.average(list(proj_num.values()))

In [ ]:
np.argmax(list(proj_num.values()))

In [ ]:
list(proj_num.keys())[213331]

In [ ]:
a = set(proj_num.keys())

In [ ]:
b = a.union(set(deps_proj.keys()))

In [ ]:
len(b)

In [ ]:
len(proj_num)

In [ ]:
res = db.python.find()
import scipy

In [ ]:
match_id = dict()

repo_pro = dict()
repo_dep = dict()

class idgiver:
    def __init__(self):
        self.iterr=0
        self.repo_pro2 = dict()
        self.repo_dep2 = dict()
        
    def get_id(self, g_id):
        if not match_id.get(g_id, False):
            match_id[g_id] = self.iterr
            self.iterr+=1
        return match_id[g_id]
idgive = idgiver()

In [ ]:
curs = db.js6.find()
for entry in curs:
    p = entry['project']
    d = entry['depenendent']
    repo_pro[p]= repo_pro.get(p, 0) + 1
    repo_dep[d]= repo_dep.get(d, 0) + 1    

In [ ]:
curs = db.js6.find()
for entry in curs:
    if repo_pro[entry['project']]>=20 and repo_dep[entry['depenendent']]>=20 and \
        repo_pro[entry['project']]<=100 and repo_dep[entry['depenendent']]<=2000:
        
        p = entry['project']
        d = entry['depenendent']
        idgive.repo_pro2[p]= idgive.repo_pro2.get(p, 0) + 1
        idgive.repo_dep2[d]= idgive.repo_dep2.get(d, 0) + 1  

In [ ]:
print(len(idgive.repo_pro2))
print(len(idgive.repo_dep2))

In [ ]:
curs = db.js6.find()
edges_list = list()
for entry in curs:

    if idgive.repo_pro2.get(entry['project'],0)>=20 and idgive.repo_dep2.get(entry['depenendent'],0)>=20:
        p = idgive.get_id(entry['project'])
        d = idgive.get_id(entry['depenendent'])
        edges_list.append((p,d))
        edges_list.append((d,p))

In [ ]:
len(edges_list)

In [ ]:
idgive.iterr

In [ ]:
import networkx as nx
G=nx.Graph()
G.add_edges_from(edges_list)

In [ ]:
for edge in G.edges():
    G[edge[0]][edge[1]]['weight'] = 1

In [ ]:
G.to_undirected()

In [ ]:
Gx = Graph(G, False, 1, 1)

In [ ]:
Gx.preprocess_transition_probs()

In [ ]:
Gx.self.alias_nodes

In [ ]:


def save_obj(obj, name ):
    with open('obj/' + name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)



In [ ]:
save_obj(match_id, 'match_id')
save_obj(id_to_name, 'id_to_name')
save_obj(edges_list, 'edges_list')
save_obj(repo_count, 'repo_count')



In [ ]:
match_id = load_obj('match_id')
id_to_name = load_obj('id_to_name')
edges_list = load_obj('edges_list')
repo_count = load_obj('repo_count')




In [ ]:
len(edges_list)

In [ ]:
col = [ k for (k,_) in edges_list]
row = [ w for (_,w) in edges_list]


In [ ]:
sha_pe = idgive.iterr
print(sha_pe)


## mid results

sp_m - sparse matrix


In [ ]:
data = len(col)*[1]
print(len(data))

In [ ]:
sp_m = sparse.coo_matrix((data, (row,col)), shape=(sha_pe,sha_pe)).tocsr()

In [ ]:
sp_m = sparse.load_npz('obj/sparse.npz')

In [ ]:
sparse.save_npz('obj/sparse4', sp_m)

In [ ]:
sp_m.indptr

In [ ]:
p = 1.0
q=0.5

#### sp_m, p, q are defined

In [ ]:
class randomWalks:
    def __init__(self, sp_m, p, q):
        self.sp_m = sp_m
        self.p = p
        self.q = q
        self.random_walks = list()
        self.transition = dict()
rW = randomWalks(sp_m, p, q)

In [ ]:
import multiprocessing as mp
cpu_count = mp.cpu_count()
from tqdm import tqdm


def compute_transition_prob(adj_mat_csr_sparse, p, q):

    transition={}
    num_nodes=adj_mat_csr_sparse.shape[0]
    indices=adj_mat_csr_sparse.indices
    indptr=adj_mat_csr_sparse.indptr
    data=adj_mat_csr_sparse.data
    #Precompute the transition matrix in advance
    for t in range(num_nodes):#t is row index
        if t%10==0:
            print('iteration: {0} '.format(t))
        for v in indices[indptr[t]:indptr[t+1]]:#i.e  possible next ndoes from t
            pi_vx_indices=indices[indptr[v]:indptr[v+1]]#i.e  possible next ndoes from v
            pi_vx_values = np.array([alpha(p,q,t,x,adj_mat_csr_sparse) for x in pi_vx_indices])
            pi_vx_values=pi_vx_values*data[indptr[v]:indptr[v+1]]

            pi_vx_values=pi_vx_values/np.sum(pi_vx_values)
            #now, we have normalzied transion probabilities for v traversed from t
            #the probabilities are stored as a sparse vector. 
            transition[t,v]=(pi_vx_indices,pi_vx_values)

    return transition


def alpha(p,q,t,x,adj_mat_csr_sparse):
    if t==x:
        return 1.0/p
    elif adj_mat_csr_sparse[t,x]>0:
        return 1.0
    else:
        return 1.0/q
    
def generate_random_walks(dfasdfds):
    adj_mat_csr_sparse=rW.sp_m
    transition=rW.transition
    random_walk_length=100
    random_walks=[]
    
    num_nodes=adj_mat_csr_sparse.shape[0]
    indices=adj_mat_csr_sparse.indices
    indptr=adj_mat_csr_sparse.indptr
    data=adj_mat_csr_sparse.data
    #get random walks
    for u in range(num_nodes):
        if len(indices[indptr[u]:indptr[u+1]]) !=0:
            #first move is just depends on weight
            possible_next_node=indices[indptr[u]:indptr[u+1]]
            weight_for_next_move=data[indptr[u]:indptr[u+1]]#i.e  possible next ndoes from u
            weight_for_next_move=weight_for_next_move.astype(np.float32)/np.sum(weight_for_next_move)
            first_walk=np.random.choice(possible_next_node, 1, p=weight_for_next_move)
            random_walk=[u,first_walk[0]]
            for i in range(random_walk_length-2):
                cur_node = random_walk[-1]
                precious_node=random_walk[-2]
#                 print(precious_node)
#                 print(cur_node)
                (pi_vx_indices,pi_vx_values)=transition[precious_node,cur_node]
                next_node=np.random.choice(pi_vx_indices, 1, p=pi_vx_values)
                random_walk.append(next_node[0])
            random_walks.append(random_walk)

    return random_walks

In [ ]:
transition = compute_transition_prob(sp_m, p, q)

In [ ]:
rW.transition = transition

In [ ]:
save_obj(transition, 'transition')

In [ ]:
transition = load_obj('transition')

In [ ]:
len(transition)

In [ ]:
random_walk_length=100

pool = mp.Pool(cpu_count)
callback = pool.map(generate_random_walks, range(cpu_count))
pool.close()


In [ ]:
random_walks = []
for entry in callback:
    random_walks.extend(entry) 


In [ ]:
len(random_walks)

In [ ]:
np_random_walks=np.array(random_walks,dtype=np.int32)
np.savez('obj/np_random_walks4',np_random_walks)

In [ ]:
np_random_walks = np.load('obj/np_random_walks.npz')['arr_0']

In [ ]:
len(np_random_walks)

In [ ]:
import numpy as np
import tensorflow as tf
import argparse
import math

In [ ]:
adj_mat_csr_sparse =sp_m

In [ ]:
np.random.shuffle(np_random_walks)

In [ ]:
num_nodes=adj_mat_csr_sparse.shape[0]
context_size=16
batch_size = None
embedding_size = 200 # Dimension of the embedding vector.
num_sampled = 64 # Number of negative examples to sample.


In [ ]:
global_step = tf.Variable(0, name='global_step', trainable=False)

# Parameters to learn
node_embeddings = tf.Variable(tf.random_uniform([num_nodes, embedding_size], -1.0, 1.0))
softmax_weights = tf.Variable(tf.truncated_normal([num_nodes, embedding_size],stddev=1.0 / math.sqrt(embedding_size)))
softmax_biases = tf.Variable(tf.zeros([num_nodes]))


In [ ]:
softmax_weights

In [ ]:

with tf.name_scope("context_node") as scope:
    #context nodes to each input node in the batch (e.g [[1,2],[4,6],[5,7]] where batch_size = 3,context_size=3)
    train_context_node= tf.placeholder(tf.int32, shape=[batch_size,context_size],name="context_node")
    #orgenize prediction labels (skip-gram model predicts context nodes (i.e labels) given a input node)
    #i.e make [[1,2,4,6,5,7]] given context above. The redundant dimention is just for restriction on tensorflow API.
    train_context_node_flat=tf.reshape(train_context_node,[-1,1])
with tf.name_scope("input_node") as scope:
    #batch input node to the network(e.g [2,1,3] where batch_size = 3)
    train_input_node= tf.placeholder(tf.int32, shape=[batch_size],name="input_node")
    #orgenize input as flat. i.e we want to make [2,2,2,1,1,1,3,3,3] given the  input nodes above
    input_ones=tf.ones_like(train_context_node)
    train_input_node_flat=tf.reshape(tf.multiply(input_ones,tf.reshape(train_input_node,[-1,1])),[-1])


# # Model.
# with tf.name_scope("loss") as scope:
#     # Look up embeddings for words.
#     node_embed = tf.nn.embedding_lookup(node_embeddings, train_input_node_flat)
#     # Compute the softmax loss, using a sample of the negative labels each time.
#     loss_node2vec = tf.reduce_mean(tf.nn.sampled_softmax_loss(softmax_weights,softmax_biases,node_embed,train_context_node_flat, num_sampled, num_nodes))
#     loss_node2vec_summary = tf.scalar_summary("loss_node2vec", loss_node2vec)


In [ ]:
num_nodes

In [ ]:
with tf.name_scope("loss") as scope:
    # Look up embeddings for words.
    node_embed = tf.nn.embedding_lookup(node_embeddings, train_input_node_flat)
    # Compute the softmax loss, using a sample of the negative labels each time.
    loss_node2vec = tf.reduce_mean(tf.nn.sampled_softmax_loss(tf.cast(softmax_weights, tf.int32),
                                                              tf.cast(softmax_biases, tf.int32),
                                                              node_embed,
                                                              train_context_node_flat,
                                                              num_sampled,
                                                              num_nodes))
    loss_node2vec_summary = tf.scalar_summary("loss_node2vec", loss_node2vec)
#

In [ ]:
train_context_node_flat